In [25]:
# url = "https://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&appKey=ETBLOG,LIFEH,ETH,MALAY,TOI,FMI,TOIRB,ZT,IEX,BI,MTB,iDiva,ETR,TELGU,QNAZ,ETT,BM,HEALTHB,TAMB,AdAge,IV,HMP,ZW,ETG,itimes,HTFOOD,GTech,ET,TAMIL,PG,TJC,TR,Travel,MYT_COMMENTS_API_KEY,NBTB,MXP,MKYRA,PM,NBTO,EA,ZEX,NPRSS,TIMESB,ETBFSI,NGSBLOG,LUXP,NPRS,QNA,ES,ELE,TFOOD,ETCIO,MTO,BT,MYTIMES,ST,TS,ESB,ETE,TD,NGS,VKB,mdfkfkf,ETHFI,MALB,CRIC,FMF,GIZ,FEMB,TMC,TELB,NBTRB,TOIBLOG,TBNET,MFB,NBTBU,TILC,MB,Test,Gaana,ETHC,ETPB,CMSA,KS,FEMINA,MM,AM,ETA,iDiva2,MALAYGN,TELGUGN,TAMILGN,VKGN,ESGN,MTGN,NBTG,BNETH,ETPRIME,INDIATIMES,MENSXP,NBTGTECH,NTOIRB,STEN,STHN,TECHGIG,TESTMYT,IAG,ETTECH,ESG&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED&sDate=1%2F1%2F2021&eDate=31%2F1%2F2021&from=0&size=0&queryString=_exists_%3AC_T&aggField=A_K&aggSize=100&aggMinDocCount=1"

In [74]:
import requests
import pandas as pd
import numpy as np
import urllib.parse

In [82]:
# S_DATE = "1%2F1%2F2021"
# E_DATE = "31%2F1%2F2021"
# S_DATE = "1%2F2%2F2021"
# E_DATE = "28%2F2%2F2021"
S_DATE = urllib.parse.quote_plus("1/2/2021")
E_DATE = urllib.parse.quote_plus("28/2/2021")

def get_elastic_url(q_str_):
    url_ = "https://commentmoderator.indiatimes.com/mytimes/elasticCommentQuery?sort=desc&filterCommentStatus=APPROVED,REJECTED,UNVERIFIED&sDate={}&eDate={}&from=0&size=0&queryString={}&aggField=A_K&aggSize=300&aggMinDocCount=1".format(S_DATE, E_DATE, q_str_)
    return url_

In [83]:
q_str_spam_rule = "_exists_:C_T%20AND%20(C_APP_RES:ARTICLE_PASTED%20OR%20C_APP_RES:BATCH_PROCESS_PAST%20OR%20C_APP_RES:DUPLICATE%20OR%20C_APP_RES:EMAIL%20OR%20C_APP_RES:LARGE_COMMENT%20OR%20C_APP_RES:OFFENSIVE%20OR%20C_APP_RES:URL%20OR%20C_APP_RES:PHONE_NUM)"
url = get_elastic_url(q_str_spam_rule)
json_res = requests.get(url).json()
dict_spam_rule = {item['key']:item['doc_count'] for item in json_res['aggregations']['2']['buckets']}

In [84]:
# dict_spam_rule

In [85]:
# dict_spam_rule

In [86]:
q_str_total_cnt = "_exists_:C_T"
url = get_elastic_url(q_str_total_cnt)
json_res = requests.get(url).json()
dict_total_cnt = {item['key']:item['doc_count'] for item in json_res['aggregations']['2']['buckets']}

In [87]:
dict_total_cnt

{'TOI': 296127,
 'ET': 29031,
 'NBTO': 20555,
 'MTO': 18025,
 'TIMESB': 4730,
 'ES': 3012,
 'MM': 2349,
 'BM': 2016,
 'NBTRB': 1852,
 'IV': 1319,
 'ESG': 1241,
 'NBTG': 830,
 'GTech': 752,
 'INDIATIMES': 580,
 'TELGU': 569,
 'TOIRB': 531,
 'STEN': 434,
 'ETBLOG': 400,
 'ETPRIME': 371,
 'TFOOD': 351,
 'MENSXP': 296,
 'PM': 214,
 'AM': 163,
 'TJC': 119,
 'TAMIL': 116,
 'iDiva2': 102,
 'Travel': 97,
 'NBTB': 61,
 'C1_clientidtesting2': 32,
 'FEMINA': 30,
 'FMI': 28,
 'MALAY': 23,
 'MTB': 12,
 'C1_clientidtesting1': 9,
 'C1_testClientId2': 7,
 'ESB': 7,
 'STHN': 6,
 'PG': 5,
 'BT': 4,
 'qatest1_testClientId2': 4,
 'ETTECH': 1,
 'HTFOOD': 1,
 'IAG': 1,
 'MKYRA': 1,
 'MYTIMES': 1}

In [31]:
q_str_ml_based = "_exists_:C_T%20AND%20C_APP_RES:ML_PROCESS"
url = get_elastic_url(q_str_ml_based)
json_res = requests.get(url).json()
dict_ml_based = {item['key']:item['doc_count'] for item in json_res['aggregations']['2']['buckets']}

In [32]:
q_str_editor_action = "_exists_:C_T%20AND%20(C_APP_RES:1%20OR%20C_APP_RES:PURGE_ACTION%20OR%20C_APP_RES:ED_OFFENSIVE)"
url = get_elastic_url(q_str_editor_action)
json_res = requests.get(url).json()
dict_editor_action = {item['key']:item['doc_count'] for item in json_res['aggregations']['2']['buckets']}

In [33]:
df1 = pd.DataFrame(index=list(dict_total_cnt.keys()), columns = ['Total Comment Count', 'Spam Rule Based Rejections Count', 'ML Based Rejected Count', 'Editor Action Rejections Count'])
df1 = df1.fillna(0)

In [34]:
# df1.index.map(dict_total_cnt)

In [35]:
# dict_total_cnt

In [36]:
df1['Total Comment Count'] = df1.index.map(dict_total_cnt)
df1['Spam Rule Based Rejections Count'] = df1.index.map(dict_spam_rule)
df1['ML Based Rejected Count'] = df1.index.map(dict_ml_based)
df1['Editor Action Rejections Count'] = df1.index.map(dict_editor_action)

In [37]:
df1 = df1.fillna(0)

In [38]:
df1['ML Based Rejected Count'] = df1['ML Based Rejected Count'].astype(np.int64)
df1['Spam Rule Based Rejections Count'] = df1['Spam Rule Based Rejections Count'].astype(np.int64)
df1['Editor Action Rejections Count'] = df1['Editor Action Rejections Count'].astype(np.int64)

In [39]:
df1

,Total Comment Count,Spam Rule Based Rejections Count,ML Based Rejected Count,Editor Action Rejections Count
TOI,296127,5225,12673,2561
ET,29031,2212,0,549
NBTO,20555,226,179,37
MTO,18025,115,0,17
TIMESB,4730,132,0,2
ES,3012,21,0,0
MM,2349,95,0,0
BM,2016,92,0,0
NBTRB,1852,23,0,0
IV,1319,7,5,3


In [48]:
df1['appkey'] = df1.index
df1['to_include'] = df1['appkey'].apply(lambda x: "_" not in x)
df1 = df1[df1['to_include']].drop(columns = ['appkey', 'to_include'])

In [51]:
MIN_COUNT_THRESHOLD = 10
df1 = df1[df1['Total Comment Count']>MIN_COUNT_THRESHOLD]

In [52]:
df1

,Total Comment Count,Spam Rule Based Rejections Count,ML Based Rejected Count,Editor Action Rejections Count
TOI,296127,5225,12673,2561
ET,29031,2212,0,549
NBTO,20555,226,179,37
MTO,18025,115,0,17
TIMESB,4730,132,0,2
ES,3012,21,0,0
MM,2349,95,0,0
BM,2016,92,0,0
NBTRB,1852,23,0,0
IV,1319,7,5,3


In [56]:
channel_desc_map = {
    'NBTO': 'NBT',
    'MTO' : 'MT',
    'TIMESB' : 'Times Blog',
    'ES' : 'EI Samay',
    'MM' : 'Mumbai Mirror',
    'BM' : 'Bangalore Mirror',
    'NBTRB' : 'NBT Readers Blog',
    'IV' : 'Indiatimes Vijaykarnataka',
    'ESG' : 'EI Samay Gold',
    'NBTG' : 'Navbharat Gold',
    'GTech' : 'GTech',
    'INDIATIMES' : 'Indiatimes',
    'TELGU' : 'Telgu',
    'TOIRB' : 'TOI Readers Blog',
    'STEN' : 'Speaking Tree English',
    'ETBLOG' : 'ET Blog',
    'ETPRIME' : 'ET Prime',
    'TFOOD' : 'Times Food',
    'MENSXP' : 'MENSXP',
    'PM' : 'Pune Mirror',
    'AM' : 'Ahmedabad Mirror',
    'TJC' : 'TimesJobs Content',
    'TAMIL' : 'Tamil',
    'iDiva2': 'iDiva2',
    'Travel': 'Travel',
    'NBTB': 'NBT Blogs',
    'FEMINA': 'FEMINA',
    'FMI': 'Femina miss india',
    'MALAY' : 'Malayalam',
    'MTB' : 'Maharashtra Times Blog'
}

In [59]:
df1['channel'] = df1.index

/Users/yash.dalmia/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [69]:
# df1

In [65]:
# channel_desc_map.get('NBTOO', 0)

In [66]:
df1['channel_desc'] = df1['channel'].apply(lambda x: channel_desc_map.get(x, x))

/Users/yash.dalmia/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [67]:
df1

,Total Comment Count,Spam Rule Based Rejections Count,ML Based Rejected Count,Editor Action Rejections Count,channel,channel_desc
TOI,296127,5225,12673,2561,TOI,TOI
ET,29031,2212,0,549,ET,ET
NBTO,20555,226,179,37,NBTO,NBT
MTO,18025,115,0,17,MTO,MT
TIMESB,4730,132,0,2,TIMESB,Times Blog
ES,3012,21,0,0,ES,EI Samay
MM,2349,95,0,0,MM,Mumbai Mirror
BM,2016,92,0,0,BM,Bangalore Mirror
NBTRB,1852,23,0,0,NBTRB,NBT Readers Blog
IV,1319,7,5,3,IV,Indiatimes Vijaykarnataka


In [68]:
# df1.to_csv('/Users/yash.dalmia/feb_monthly_moderation_report.csv', encoding='utf-8')
df1.to_csv('feb_monthly_moderation_report.csv', encoding='utf-8')

In [22]:
# client_channels_to_remove = ['C1_testClientId1', 'C1_testClientId2', 'qatest1_testClientId2', 'qatest2_testClientId2']